In [ ]:
import gzip
import io
import pandas as pd
import os
from collections import Counter
from umi_tools import UMIClusterer
import matplotlib.pyplot as plt
from Bio import SeqIO
from collections import Counter
import csv
import re
import numpy as np
from scipy.stats import pearsonr
import seaborn as sns
from adjustText import adjust_text  # Import the adjust_text function
from matplotlib.lines import Line2D  # Import Line2D
from brokenaxes import brokenaxes
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import LinearSegmentedColormap
%matplotlib inline

In [ ]:
# loading files
path_to_files = "/corgi/martin/CRISPR_padlock/BK_FINAL_DATA/5_Johan_files/"

p_umi_mle = pd.read_csv(path_to_files+"14d_mip_dedup#allreads#0.mageckmle.gene_summary.txt", sep='\t')
p_mle = pd.read_csv(path_to_files+"14d_mip_nodedup#allreads#0.mageckmle.gene_summary.txt", sep='\t')
PCR_mle = pd.read_csv(path_to_files+"14d_pcr#allreads#0.mageckmle.gene_summary.txt", sep='\t')

p_umi_test = pd.read_csv(path_to_files+"14d_mip_dedup#allreads#0.gene_summary.txt", sep='\t')
p_test = pd.read_csv(path_to_files+"14d_mip_nodedup#allreads#0.gene_summary.txt", sep='\t')
PCR_test = pd.read_csv(path_to_files+"14d_pcr#allreads#0.gene_summary.txt", sep='\t')

### Figure 3A

In [ ]:
# MAGeCK test
def plot_RRA_score2(df, gene_list, name):
    # Filter genes with 'eff|fdr' value < 0.05
    highlighted_genes = df[df['neg|fdr'] < 0.05]

    # Identify genes in the gene_list
    green_genes_mask = df['id'].isin(gene_list)

    # Create the scatter plot for 'neg|score' with a logarithmic scale on the y-axis
    plt.figure(figsize=(6, 6))
    # All genes in dark grey with black edge
    plt.scatter(range(len(df)), df['neg|score'], alpha=0.7, color='dimgrey', edgecolor='dimgrey', label='All Genes', s=20)
    
    # Set logarithmic scale on y-axis
    plt.yscale('log')
    
    # Set the y-axis range if specified
    plt.ylim(0.0000001, 2)

    # Highlight filtered genes in red with black edge, excluding those in gene_list to prioritize green
    plt.scatter(highlighted_genes[~highlighted_genes['id'].isin(gene_list)].index, highlighted_genes[~highlighted_genes['id'].isin(gene_list)]['neg|score'], color='red', edgecolor='black', label='Highlighted Genes (< 0.05 eff|fdr)',s=50)

    # Highlight genes from gene_list in green
    plt.scatter(df[green_genes_mask].index, df[green_genes_mask]['neg|score'], color='green', edgecolor='black', label='Gene List Highlight', s=50)

    # Initialize a list to store text objects for adjust_text
    texts = []

    # Annotate genes in the gene_list with green text
    for _, row in df[green_genes_mask].iterrows():
        texts.append(plt.text(row.name, row['neg|score'], row['id'], ha='center', va='bottom', color='green'))

    #  Use adjust_text to repel labels with stronger repulsion parameters
    adjust_text(texts, arrowprops=dict(arrowstyle='-', color='black'),
                expand_points=(2.5, 2.5), expand_text=(1.5, 1.5),
                force_points=0.5, force_text=0.5)

    # Set labels and title
    plt.xlabel('Gene Rank', fontsize=14)
    plt.ylabel('RRA Score', fontsize=14)
    plt.title('RRA Score by Gene Rank')

    # Reverse values on y-axis
    plt.gca().invert_yaxis()

    # Increase the thickness of the axis lines and tick marks
    ax = plt.gca()
    ax.spines['top'].set_linewidth(2)
    ax.spines['bottom'].set_linewidth(2)
    ax.spines['left'].set_linewidth(2)
    ax.spines['right'].set_linewidth(2)
    ax.tick_params(axis='both', which='major', labelsize=12, width=2, length=6)

    # save plot as svg
    plt.savefig(f'/corgi/martin/CRISPR_padlock/BK_FINAL_DATA/5_Johan_files/figures/{name}.svg', format='svg')
    
    # save plot as png
    plt.savefig(f'/corgi/martin/CRISPR_padlock/BK_FINAL_DATA/5_Johan_files/figures/{name}.png', format='png', dpi=600) 
    
    # Show plot
    plt.tight_layout()
    plt.show()
    
gene_list = ["GUK1", "AURKB", "WEE1", "DOLK"]
plot_RRA_score2(PCR_test,gene_list,"RRA_PCR")
plot_RRA_score2(p_umi_test,gene_list,"RRA_padlock_umi")

# MAGeCK mle
def plot_beta_score_genes2(df, gene_list, name):
    # Filter genes with 'eff|fdr' value < 0.05
    highlighted_genes = df[df['eff|fdr'] < 0.05]

    # Create the scatter plot for 'eff|beta' with a logarithmic scale on the y-axis
    plt.figure(figsize=(6, 6))
    # All genes in dark grey with black edge
    plt.scatter(range(len(df)), df['eff|beta'], alpha=0.7, color='dimgrey', edgecolor='dimgrey', label='All Genes', s=20)

    # Highlight filtered genes in red with black edge, excluding those in gene_list to prioritize green
    plt.scatter(highlighted_genes[~highlighted_genes['id'].isin(gene_list)].index, highlighted_genes[~highlighted_genes['id'].isin(gene_list)]['eff|beta'], color='red', edgecolor='black', label='Highlighted Genes (< 0.05 eff|fdr)',s=50)

    # Highlight genes from gene_list in green
    plt.scatter(df[df['id'].isin(gene_list)].index, df[df['id'].isin(gene_list)]['eff|beta'], color='green', edgecolor='black', label='Gene List Highlight', s=50)

    # Initialize a list to store text objects for adjust_text
    texts = []

    # Annotate genes in the gene_list and add them to the texts list
    for _, row in df[df['id'].isin(gene_list)].iterrows():
        texts.append(plt.text(row.name, row['eff|beta'], row['id'], ha='center', va='bottom',color='green'))

    #  Use adjust_text to repel labels with stronger repulsion parameters
    adjust_text(texts, arrowprops=dict(arrowstyle='-', color='black'),
                expand_points=(2.5, 2.5), expand_text=(1.5, 1.5),
                force_points=0.5, force_text=0.5)

    # Set labels and title
    plt.xlabel('Gene Rank', fontsize=14)
    plt.ylabel('Beta Score', fontsize=14)
    plt.title('Beta Score by Gene Rank')
    
    # Set the y-axis range if specified
    plt.ylim(-2.1, 0.8)
        
    # Increase the thickness of the axis lines
    ax = plt.gca()
    ax.spines['top'].set_linewidth(2)
    ax.spines['bottom'].set_linewidth(2)
    ax.spines['left'].set_linewidth(2)
    ax.spines['right'].set_linewidth(2)
    
    # Increase the thickness and size of scale labels and tick marks
    ax.tick_params(axis='both', which='major', labelsize=12, width=2, length=6) 
    
    # save plot as svg
    plt.savefig(f'/corgi/martin/CRISPR_padlock/BK_FINAL_DATA/5_Johan_files/figures/{name}.svg', format='svg')
    
    # save plot as png
    plt.savefig(f'/corgi/martin/CRISPR_padlock/BK_FINAL_DATA/5_Johan_files/figures/{name}.png', format='png', dpi=600) 

    # Show plot
    plt.tight_layout()
    plt.show()
    
plot_beta_score_genes2(p_umi_mle, gene_list, "beta_padlock_umi")
plot_beta_score_genes2(PCR_mle, gene_list, "beta_PCR")

### Figure 3B

In [2]:
# MAGecK test
def correlation_rank_grad(df1, df2, gene_list, name):
    # Merge the DataFrames on the 'id' column, including 'neg|fdr' columns
    common_genes = pd.merge(df1[['id', 'neg|rank', 'neg|fdr']], df2[['id', 'neg|rank', 'neg|fdr']], on='id', suffixes=('_df1', '_df2'))

    # Calculate the product of 'neg|fdr' values from df1 and df2 for the color gradient
    score_product = common_genes['neg|fdr_df1'] * common_genes['neg|fdr_df2']

    # Create scatter plot with gradient colors based on the score product using the red-white-blue color map
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(common_genes['neg|rank_df1'], common_genes['neg|rank_df2'], c=score_product, cmap=sns.dark_palette("#69d", reverse=True, as_cmap=True), s=15)

    # Add a color bar to the plot to represent the scale of the score product
    plt.colorbar(scatter, label='Product of neg|fdr')

    # Highlight genes from gene_list in green
    green_genes_mask = common_genes['id'].isin(gene_list)
    plt.scatter(common_genes[green_genes_mask]['neg|rank_df1'], common_genes[green_genes_mask]['neg|rank_df2'], color='green', edgecolor='black', s=40)

    # Calculate Pearson correlation coefficient for 'neg|rank' columns
    corr_coefficient, _ = pearsonr(common_genes['neg|rank_df1'], common_genes['neg|rank_df2'])
    corr_coefficient = round(corr_coefficient, 2)
    print(f"Pearson Correlation Coefficient: {corr_coefficient}")

    # Fit a linear regression line to the 'neg|rank' data
    m, b = np.polyfit(common_genes['neg|rank_df1'], common_genes['neg|rank_df2'], 1)
    x_values = np.linspace(common_genes['neg|rank_df1'].min(), common_genes['neg|rank_df1'].max(), 100)
    y_values = m * x_values + b
    plt.plot(x_values, y_values, '-', color='red', label=f'Linear Regression (r={corr_coefficient})')

    plt.xlabel('neg|rank_df1')
    plt.ylabel('neg|rank_df2')
    plt.title('Visualization of neg|rank values for Common Genes')

    # Increase the thickness of the axis lines and tick marks
    ax = plt.gca()
    ax.spines['top'].set_linewidth(2)
    ax.spines['bottom'].set_linewidth(2)
    ax.spines['left'].set_linewidth(2)
    ax.spines['right'].set_linewidth(2)
    ax.tick_params(axis='both', which='major', labelsize=12, width=2, length=6)
    
    # save plot as svg
    plt.savefig(f'/corgi/martin/CRISPR_padlock/BK_FINAL_DATA/5_Johan_files/figures/{name}.svg', format='svg')
    
    # save plot as png
    plt.savefig(f'/corgi/martin/CRISPR_padlock/BK_FINAL_DATA/5_Johan_files/figures/{name}.png', format='png', dpi=600) 

    plt.show()
    
gene_list = ["GUK1", "AURKB", "WEE1", "DOLK"]
correlation_rank_grad(p_umi_test, PCR_test,gene_list, "test_corr")

# MAGecK mle
def correlation_rank_grad(df1, df2, gene_list, name):
    # Merge the DataFrames on the 'id' column, including 'neg|fdr' columns
    common_genes = pd.merge(df1[['id', 'neg|rank', 'eff|fdr']], df2[['id', 'neg|rank', 'eff|fdr']], on='id', suffixes=('_df1', '_df2'))

    # Calculate the product of 'neg|fdr' values from df1 and df2 for the color gradient
    score_product = common_genes['eff|fdr_df1'] * common_genes['eff|fdr_df2']

    # Create scatter plot with gradient colors based on the score product using the red-white-blue color map
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(common_genes['neg|rank_df1'], common_genes['neg|rank_df2'], c=score_product, cmap=sns.dark_palette("#69d", reverse=True, as_cmap=True), s=15)

    # Add a color bar to the plot to represent the scale of the score product
    plt.colorbar(scatter, label='Product of neg|fdr')

    # Highlight genes from gene_list in green
    green_genes_mask = common_genes['id'].isin(gene_list)
    plt.scatter(common_genes[green_genes_mask]['neg|rank_df1'], common_genes[green_genes_mask]['neg|rank_df2'], color='green', edgecolor='black', s=40)

    # Calculate Pearson correlation coefficient for 'neg|rank' columns
    corr_coefficient, _ = pearsonr(common_genes['neg|rank_df1'], common_genes['neg|rank_df2'])
    corr_coefficient = round(corr_coefficient, 2)
    print(f"Pearson Correlation Coefficient: {corr_coefficient}")

    # Fit a linear regression line to the 'neg|rank' data
    m, b = np.polyfit(common_genes['neg|rank_df1'], common_genes['neg|rank_df2'], 1)
    x_values = np.linspace(common_genes['neg|rank_df1'].min(), common_genes['neg|rank_df1'].max(), 100)
    y_values = m * x_values + b
    plt.plot(x_values, y_values, '-', color='red', label=f'Linear Regression (r={corr_coefficient})')

    plt.xlabel('neg|rank_df1')
    plt.ylabel('neg|rank_df2')
    plt.title('Visualization of neg|rank values for Common Genes')

    # Increase the thickness of the axis lines and tick marks
    ax = plt.gca()
    ax.spines['top'].set_linewidth(2)
    ax.spines['bottom'].set_linewidth(2)
    ax.spines['left'].set_linewidth(2)
    ax.spines['right'].set_linewidth(2)
    ax.tick_params(axis='both', which='major', labelsize=12, width=2, length=6)
    
    # save plot as svg
    plt.savefig(f'/corgi/martin/CRISPR_padlock/BK_FINAL_DATA/5_Johan_files/figures/{name}.svg', format='svg')
    
    # save plot as png
    plt.savefig(f'/corgi/martin/CRISPR_padlock/BK_FINAL_DATA/5_Johan_files/figures/{name}.png', format='png', dpi=600) 

    plt.show()
    
gene_list = ["GUK1", "AURKB", "WEE1", "DOLK"]
correlation_rank_grad(p_umi_mle, PCR_mle,gene_list, "mle_corr")

### Figure 3C

In [3]:
# load files
sgrna_p = pd.read_csv(path_to_files+"14d_mip_dedup#allreads#0.sgrna_summary.txt", sep='\t')
sgrna_PCR = pd.read_csv(path_to_files+"14d_pcr#allreads#0.sgrna_summary.txt", sep='\t')

In [ ]:
def create_summary_df(df, gene_list):
    # Initialize the new DataFrame
    summary_df = pd.DataFrame(columns=["Gene", "control_count", "treatment_count"])
    
    # Dictionary to keep track of the count for each gene
    gene_counts = {gene: 0 for gene in gene_list}

    # Iterate through the DataFrame
    for _, row in df.iterrows():
        gene = row['Gene']
        # Check if the gene is in the gene_list
        if gene in gene_list:
            # Update the count for this gene
            gene_counts[gene] += 1
            # Modify the gene name to include the count
            gene_name = f"{gene}_{gene_counts[gene]}"

            # Calculate the mean of values in 'control_count' and 'treatment_count', assuming they are separated by "/"
            control_counts = [float(x) for x in row['control_count'].split('/')]
            treatment_counts = [float(x) for x in row['treatment_count'].split('/')]
            control_mean = sum(control_counts) / len(control_counts)
            treatment_mean = sum(treatment_counts) / len(treatment_counts)

            # Add the calculated data to the summary DataFrame
            summary_df = summary_df.append({
                "Gene": gene_name,
                "control_count": control_mean,
                "treatment_count": treatment_mean
            }, ignore_index=True)
    
    # Sort the summary DataFrame by the 'Gene' column
    summary_df = summary_df.sort_values(by="Gene")
    
    return summary_df

gene_list = ["CDK1", "DOLK", "GUK1", "AURKB", "WEE1"]
padlock = create_summary_df(sgrna_p, gene_list)
PCR = create_summary_df(sgrna_PCR, gene_list)

def plot_heatmap(df,name,max_value):
    # Create a new DataFrame with genes as columns and 'Control' and 'Treatment' as index
    reshaped_df = pd.DataFrame(index=['Control', 'Treatment'])

    for gene in df['Gene']:
        reshaped_df[gene] = [df[df['Gene'] == gene]['control_count'].values[0], 
                             df[df['Gene'] == gene]['treatment_count'].values[0]]

    # Plotting the heatmap
    plt.figure(figsize=(10, 2))  # Adjust the size as needed
    ax = sns.heatmap(reshaped_df, annot=False, vmin=0, vmax=max_value, cmap="coolwarm", cbar=True, linewidths=0.3, linecolor='black', robust=True, fmt='g')
    
    # Customizing the colorbar
    cbar = ax.collections[0].colorbar
    # Set the number of ticks in the colorbar
    cbar.locator = MaxNLocator(nbins=5)  # Adjust 'nbins' to change the number of ticks
    cbar.update_ticks()
    
    plt.title('Control vs. Treatment Counts per Gene')
    plt.ylabel('Condition')
    plt.xticks(rotation=45)  # Rotate gene names for better readability

    plt.tight_layout()
    plt.savefig(f'/corgi/martin/CRISPR_padlock/BK_FINAL_DATA/5_Johan_files/figures/{name}.svg', format='svg')
    plt.show()
    
def plot_heatmap2(df,name):
    # Create a new DataFrame with genes as columns and 'Control' and 'Treatment' as index
    reshaped_df = pd.DataFrame(index=['CRISPR-MIP', 'PCR'])

    for gene in df['Gene']:
        reshaped_df[gene] = [df[df['Gene'] == gene]['CRISPR-MIP'].values[0], 
                             df[df['Gene'] == gene]['PCR'].values[0]]

    # Plotting the heatmap
    plt.figure(figsize=(10, 2))  # Adjust the size as needed
    ax = sns.heatmap(reshaped_df, annot=False, cmap="coolwarm", cbar=True, linewidths=0.05, linecolor='grey', vmin=-3.5, vmax=3.5, robust=True, fmt='g')
    
    # Customizing the colorbar
    cbar = ax.collections[0].colorbar
    # Set the number of ticks in the colorbar
    cbar.locator = MaxNLocator(nbins=5)  # Adjust 'nbins' to change the number of ticks
    cbar.update_ticks()
    
    plt.title('Control vs. Treatment Counts per Gene')
    plt.ylabel('Condition')
    plt.xticks(rotation=45)  # Rotate gene names for better readability

    plt.tight_layout()
    #plt.savefig(f'/corgi/martin/CRISPR_padlock/BK_FINAL_DATA/5_Johan_files/figures/{name}.svg', format='svg')
    plt.savefig(f'/corgi/martin/CRISPR_padlock/BK_FINAL_DATA/5_Johan_files/figures/{name}.png', format='png', dpi=600) 
    plt.show()

plot_heatmap(padlock,"heatmap_sgrnas_padlock",450)
plot_heatmap(PCR,"heatmap_sgrnas_PCR", 6000)